In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage.morphology import skeletonize
from skimage.filters import threshold_local
from tqdm import tqdm
from scipy import ndimage
from skimage import morphology
from skimage.filters import threshold_otsu

class PulmonaryVascularModel:
    def __init__(self, mode='wild_type', initial_diff_cells=0):
        self.mode = mode  # 'wild_type' или 'transgenic'
        self.initial_diff_cells = initial_diff_cells  # Количество изначально дифференцированных клеток

        # Параметры модели
        self.N = 200
        self.dx = 0.25
        self.dt = 0.0045
        self.total_steps = 1000000
        self.plot_interval = 500
        self.stability_check_interval = 100

        # Параметры уравнений
        self.c = 0.0025
        self.c0 = 0.002 #0.0025
        self.mu =0.12 #0.16 #0.18 #0.12
        self.nu = 0.04
        self.gamma = 0.02
        self.epsilon =5 # 1.5# 0.042 #01.5 #0.02
        self.d = 0.008 #0.0033
        self.sigma = 0.1
        self.f = 10

        # Диффузионные коэффициенты
        self.DA = 0.02
        self.DH =0.18 #0.26# 0.32 #0.18
        self.DS = 0.06

        # Параметры продукции
        if mode == 'wild_type':
            self.ρA = 0.03
            self.ρH = 0.0001 #0.00005
        else:
            self.ρA = 0.03
            self.ρH = 0.0003

        np.random.seed(42)
        # Параметры начальных условий
        self.A0 =1
        self.H0 = 1
        self.Y0 = 0.15
        self.A_fl = 0.02
        self.H_fl = 0.02
        self.Y_fl = 0.02
        self.noise_fractionA = 0.5
        self.noise_fractionH = 0.5
        self.noise_fractionY = 0.1

        # Инициализация переменных
        self.reset()

        self.step_count = 0

    # Сброс всех переменных к начальным значениям
    def reset(self):
        self.A = self.A0 * np.ones((self.N, self.N))
        self.H = self.H0 * np.ones((self.N, self.N))
        self.Y = self.Y0 * np.ones((self.N, self.N))
        self.S = np.ones((self.N, self.N)) * self.c0
        self.fixed_cells = np.zeros((self.N, self.N), dtype=bool)

        # Применение шума
        A_noise_mask = np.random.random(size=(self.N, self.N)) < self.noise_fractionA
        H_noise_mask = np.random.random(size=(self.N, self.N)) < self.noise_fractionH
        Y_noise_mask = np.random.random(size=(self.N, self.N)) < self.noise_fractionY

        self.A[A_noise_mask] += (2 * np.random.random(size=(self.N, self.N))[A_noise_mask] - 1) * self.A_fl * self.A0
        self.H[H_noise_mask] += (2 * np.random.random(size=(self.N, self.N))[H_noise_mask] - 1) * self.H_fl * self.H0
        self.Y[Y_noise_mask] += (2 * np.random.random(size=(self.N, self.N))[Y_noise_mask] - 1) * self.Y_fl * self.Y0

        # Добавляем изначально дифференцированные клетки
        if self.initial_diff_cells > 0:
            # Создаем случайные позиции для дифференцированных клеток
            total_cells = self.N * self.N
            indices = np.random.choice(total_cells, size=self.initial_diff_cells, replace=False)
            rows, cols = np.unravel_index(indices, (self.N, self.N))

            # Устанавливаем значения Y=1 и фиксируем эти клетки
            self.Y[rows, cols] = 1.0
            self.fixed_cells[rows, cols] = True



    def laplacian(self, Z):
        # Расчёт лапласиана через конечные разности
        Ztop = Z[0:-2, 1:-1]
        Zleft = Z[1:-1, 0:-2]
        Zbottom = Z[2:, 1:-1]
        Zright = Z[1:-1, 2:]
        Zcenter = Z[1:-1, 1:-1]
        return (Ztop + Zleft + Zbottom + Zright - 4*Zcenter) / (self.dx**2 + 1e-10)

    # Функция реакции для A (BMP-4)
    def f_A(self, A, H, S, Y):
        A_sq = A**2
        H_safe = np.maximum(H, 1e-10)
        return (self.c * A_sq * S / H_safe - self.mu * A + self.ρA * Y)

    # Функция реакции для H (MGP)
    def f_H(self, A, H, S, Y):
        A_sq = A**2
        return (self.c * A_sq * S - self.nu * H + self.ρH * Y)

    # Функция реакции для S (ALK1-TGF-β)
    def f_S(self, S, Y):
        return (self.c0 - self.gamma * S - self.epsilon * Y * S)

    # Функция реакции для Y
    def f_Y(self, A, Y):
        return (self.d * A - self.sigma * Y + (Y**2) / (1 + self.f * Y**2))

    # Улучшенный метод контрастирования с морфологической постобработкой
    def _enhance_contrast(self, data):
        # 1. Медианная фильтрация для подавления шума
        filtered = ndimage.median_filter(data, size=3)

        # 2. Нормализация на основе устойчивых статистик (игнорируем выбросы)
        p5, p95 = np.percentile(filtered, [5, 95])
        normalized = (filtered - p5) / (p95 - p5 + 1e-10)
        normalized = np.clip(normalized, 0, 1)

        # 3. Автоматический выбор порога (Оцу или фиксированный)
        if normalized.std() > 0.1:  # Достаточный контраст для метода Оцу
            threshold = threshold_otsu(normalized)
        else:
            threshold = 0.3  # Значение по умолчанию

        binary = normalized > threshold

        # 4. Морфологическая постобработка
        binary = morphology.binary_opening(binary, footprint=np.ones((2,2)))
        binary = morphology.remove_small_objects(binary, min_size=8)
        binary = morphology.remove_small_holes(binary, area_threshold=8)

        return binary

    # Шаг моделирования с необратимой дифференцировкой клеток
    def update(self):
        # 1. Вычисляем лапласианы для диффундирующих переменных
        lap_A = self.laplacian(self.A)
        lap_H = self.laplacian(self.H)
        lap_S = self.laplacian(self.S)

        # 2. Обновляем A, H, S (как раньше)
        self.A[1:-1, 1:-1] += (self.DA * lap_A * self.dt +
                              self.f_A(self.A[1:-1, 1:-1],
                                      self.H[1:-1, 1:-1],
                                      self.S[1:-1, 1:-1],
                                      self.Y[1:-1, 1:-1]) * self.dt)

        self.H[1:-1, 1:-1] += (self.DH * lap_H * self.dt +
                              self.f_H(self.A[1:-1, 1:-1],
                                      self.H[1:-1, 1:-1],
                                      self.S[1:-1, 1:-1],
                                      self.Y[1:-1, 1:-1]) * self.dt)

        self.S[1:-1, 1:-1] += (self.DS * lap_S * self.dt +
                              self.f_S(self.S[1:-1, 1:-1],
                                      self.Y[1:-1, 1:-1]) * self.dt)

        # 3. Обновляем Y ТОЛЬКО в нефиксированных ячейках
        updatable_mask = ~self.fixed_cells[1:-1, 1:-1]  # Маска для изменяемых ячеек
        Y_updates = self.f_Y(self.A[1:-1, 1:-1],
                            self.Y[1:-1, 1:-1]) * self.dt

        self.Y[1:-1, 1:-1][updatable_mask] += Y_updates[updatable_mask]

        # 4. Фиксируем новые дифференцированные клетки (Y > 0.5)
        new_diff_cells = (self.Y > 0.5) & (~self.fixed_cells)
        self.Y[new_diff_cells] = 1.0                     # Бинаризуем
        self.fixed_cells[new_diff_cells] = True          # Запоминаем навсегда

        # 5. Граничные условия (включая fixed_cells!)
        for var in [self.A, self.H, self.S, self.Y, self.fixed_cells]:
            var[0, :] = var[1, :]    # Верхняя граница
            var[-1, :] = var[-2, :]  # Нижняя граница
            var[:, 0] = var[:, 1]     # Левая граница
            var[:, -1] = var[:, -2]   # Правая граница

        self.step_count += 1

    # Процент площади с дифференцированными клетками (Y >= 0.5)
    def calculate_coverage(self):
        return np.mean(self.Y >= 0.5) * 100

    # Подсчет количества ветвлений по бинарному Y
    def count_branches(self):
        binary_Y = (self.Y >= 0.5).astype(int)
        skeleton = skeletonize(binary_Y)
        kernel = np.ones((3,3))
        neighbors = convolve2d(skeleton, kernel, mode='same') - skeleton
        branch_points = (neighbors >= 3) & skeleton
        return np.sum(branch_points)

    # Визуализация с степенным преобразованием для A, H, S
    def plot_state(self, step=None):
        plt.figure(figsize=(15, 10))
        title_suffix = f" - {self.mode}" + (f" (шаг {step})" if step else "")

        # Параметры визуализации
        gamma = 0.3  # Чем меньше gamma (например, 0.1-0.5), тем сильнее усиливаются малые значения

        # Функция для отображения с степенным преобразованием
        def power_imshow(data, ax, title, cmap='viridis'):
            # Степенное преобразование и настройка диапазона
            plot_data = data**gamma
            vmin = np.percentile(data, 1)**gamma  # 1-й процентиль для нижней границы
            vmax = np.percentile(data, 99)**gamma  # 99-й процентиль для верхней границы
            im = ax.imshow(plot_data, cmap=cmap, vmin=vmin, vmax=vmax)
            ax.set_title(f"{title} (^{gamma})")
            cbar = plt.colorbar(im, ax=ax)
            cbar.set_label(f'value^{gamma}')
            # Добавляем контуры для ясности
            ax.contour(data, levels=10, colors='white', linewidths=0.3, alpha=0.5)

        # A, H, S со степенным преобразованием
        plt.subplot(2, 3, 2)
        power_imshow(self.A, plt.gca(), f'BMP-4 (A){title_suffix}', 'plasma')

        plt.subplot(2, 3, 1)
        power_imshow(self.H, plt.gca(), f'MGP (H){title_suffix}', 'viridis')

        plt.subplot(2, 3, 3)
        power_imshow(self.S, plt.gca(), f'S (ALK1-TGF-β){title_suffix}', 'inferno')

        # Бинарные представления (без изменений)
        plt.subplot(2, 3, 5)
        plt.imshow(self._enhance_contrast(self.A), cmap='binary')
        plt.title('Бинарный A')

        plt.subplot(2, 3, 4)
        plt.imshow(self._enhance_contrast(self.H), cmap='binary')
        plt.title('Бинарный H')

        plt.subplot(2, 3, 6)
        plt.imshow(self.Y >= 0.5, cmap='binary')
        plt.title('Дифференцированные клетки')

        plt.tight_layout()
        plt.show()


    # Запуск моделирования до достижения заданного процента дифференцировки или max_steps
    def run_simulation(self, max_steps=None):
        if max_steps is None:
            max_steps = self.total_steps

        for step in tqdm(range(max_steps), desc=f"Моделирование {self.mode}"):
            self.update()

            # Проверка достижения 100% дифференцировки
            coverage = self.calculate_coverage()
            if coverage >= 40:
                print(f"\nДостигнуто заданный процент дифференцировки на шаге {step}")
                break

            # Вывод состояния и графиков с заданным интервалом
            if step % self.plot_interval == 0:
                print(f"\nШаг {step}:")
                # print("Пример фрагмента A (5x5):")
                # print(self.A[:5, :5])
                # print("Пример фрагмента H (5x5):")
                # print(self.H[:5, :5])
                # print("Пример фрагмента Y (5x5):")
                # print(self.Y[:5, :5])
                print(f"Дифференцировано клеток: {coverage:.1f}%")
                print(f"Фиксировано клеток: {np.sum(self.fixed_cells)}/{self.N*self.N}")
                # Визуализация
                self.plot_state(step=step)

        else:
            print(f"\nДостигнуто максимальное количество шагов ({max_steps})")
            print(f"Финальное покрытие: {coverage:.1f}%")

        print(f"\nФинальные результаты ({self.mode}):")
        print(f"- Покрытие дифференцированными клетками: {coverage:.1f}%")
        print(f"- Количество ветвлений: {self.count_branches()}")
        self.plot_state(step=step)

        return step  # Возвращаем количество выполненных шагов

# Сравнение wild_type и transgenic моделей с улучшенной визуализацией
def compare_models():
    # Создаем wild_type модель
    wild = PulmonaryVascularModel(mode='wild_type')
    print("Запуск wild_type модели до n% дифференцировки...")
    wild_steps = wild.run_simulation()

    # Создаем transgenic модель и запускаем на то же количество шагов
    transgenic = PulmonaryVascularModel(mode='transgenic')
    transgenic.reset()
    print(f"\nЗапуск transgenic модели на {wild_steps} шагов...")
    transgenic.run_simulation(max_steps=wild_steps)

    # Сравнительные метрики
    print("\nСравнительные результаты:")
    print(f"{'Метрика':<30} {'Wild Type':<15} {'Transgenic':<15}")
    print("-"*60)
    print(f"{'Покрытие (%)':<30} {wild.calculate_coverage():<15.1f} {transgenic.calculate_coverage():<15.1f}")
    print(f"{'Количество ветвлений':<30} {wild.count_branches():<15} {transgenic.count_branches():<15}")
    print(f"{'Шагов выполнено':<30} {wild_steps:<15} {wild_steps:<15}")

    # Рассчитываем общие пределы для цветовых шкал
    gamma = 0.3

    # Для активатора (A)
    a_min = min(wild.A.min(), transgenic.A.min())**gamma
    a_max = max(wild.A.max(), transgenic.A.max())**gamma

    # Для ингибитора (H)
    h_min = min(wild.H.min(), transgenic.H.min())**gamma
    h_max = max(wild.H.max(), transgenic.H.max())**gamma

    # Для S (ALK1-TGF-β)
    s_min = min(wild.S.min(), transgenic.S.min())**gamma
    s_max = max(wild.S.max(), transgenic.S.max())**gamma

    # Функция для создания субплотов с улучшенной визуализацией и общей шкалой
    def plot_enhanced(data, ax, title, cmap='viridis', vmin=None, vmax=None):
        plot_data = data**gamma
        im = ax.imshow(plot_data, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_title(title)
        ax.contour(data, levels=10, colors='white', linewidths=0.3, alpha=0.5)
        return im

    # Создаем большую фигуру для всех графиков
    plt.figure(figsize=(20, 25))

    # 1. Ингибитор (H) - первый ряд
    plt.subplot(6, 2, 1)
    im_h_wild = plot_enhanced(wild.H, plt.gca(), 'Wild Type - MGP (H)', 'viridis', h_min, h_max)
    plt.colorbar(im_h_wild)
    # plt.annotate(f'Min: {wild.H.min():.2f}\nMax: {wild.H.max():.2f}',
    #              xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    plt.subplot(6, 2, 2)
    im_h_trans = plot_enhanced(transgenic.H, plt.gca(), 'Transgenic - MGP (H)', 'viridis', h_min, h_max)
    plt.colorbar(im_h_trans)
    # plt.annotate(f'Min: {transgenic.H.min():.2f}\nMax: {transgenic.H.max():.2f}',
    #              xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    # 2. Бинаризованный ингибитор (H) - второй ряд
    plt.subplot(6, 2, 3)
    plt.imshow(wild._enhance_contrast(wild.H), cmap='binary')
    plt.title('Wild Type - Бинарный MGP (H)')

    plt.subplot(6, 2, 4)
    plt.imshow(transgenic._enhance_contrast(transgenic.H), cmap='binary')
    plt.title('Transgenic - Бинарный MGP (H)')

    # 3. Активатор (A) - третий ряд
    plt.subplot(6, 2, 5)
    im_a_wild = plot_enhanced(wild.A, plt.gca(), 'Wild Type - BMP-4 (A)', 'plasma', a_min, a_max)
    plt.colorbar(im_a_wild)
    # plt.annotate(f'Min: {wild.A.min():.2f}\nMax: {wild.A.max():.2f}',
    #              xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    plt.subplot(6, 2, 6)
    im_a_trans = plot_enhanced(transgenic.A, plt.gca(), 'Transgenic - BMP-4 (A)', 'plasma', a_min, a_max)
    plt.colorbar(im_a_trans)
    # plt.annotate(f'Min: {transgenic.A.min():.2f}\nMax: {transgenic.A.max():.2f}',
    #              xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    # 4. Бинаризованный активатор (A) - четвертый ряд
    plt.subplot(6, 2, 7)
    plt.imshow(wild._enhance_contrast(wild.A), cmap='binary')
    plt.title('Wild Type - Бинарный BMP-4 (A)')

    plt.subplot(6, 2, 8)
    plt.imshow(transgenic._enhance_contrast(transgenic.A), cmap='binary')
    plt.title('Transgenic - Бинарный BMP-4 (A)')

    # 5. S (ALK1-TGF-β) - пятый ряд
    plt.subplot(6, 2, 9)
    im_s_wild = plot_enhanced(wild.S, plt.gca(), 'Wild Type - ALK1-TGF-β (S)', 'inferno', s_min, s_max)
    plt.colorbar(im_s_wild)
    # plt.annotate(f'Min: {wild.S.min():.2f}\nMax: {wild.S.max():.2f}',
                 #xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    plt.subplot(6, 2, 10)
    im_s_trans = plot_enhanced(transgenic.S, plt.gca(), 'Transgenic - ALK1-TGF-β (S)', 'inferno', s_min, s_max)
    plt.colorbar(im_s_trans)
    # plt.annotate(f'Min: {transgenic.S.min():.2f}\nMax: {transgenic.S.max():.2f}',
                 #xy=(0.05, 0.05), xycoords='axes fraction', color='white')

    # 6. Бинаризованный S (ALK1-TGF-β) - шестой ряд
    plt.subplot(6, 2, 11)
    plt.imshow(wild._enhance_contrast(wild.S), cmap='binary')
    plt.title('Wild Type - Бинарный ALK1-TGF-β (S)')

    plt.subplot(6, 2, 12)
    plt.imshow(transgenic._enhance_contrast(transgenic.S), cmap='binary')
    plt.title('Transgenic - Бинарный ALK1-TGF-β (S)')

    plt.tight_layout()
    plt.show()

    # Отдельная фигура для дифференцировки
    plt.figure(figsize=(15, 7))

    plt.subplot(1, 2, 1)
    plt.imshow(wild.Y >= 0.5, cmap='binary')
    plt.title(f'Wild Type - Дифференцировка (покрытие: {wild.calculate_coverage():.1f}%, ветвления: {wild.count_branches()})')

    plt.subplot(1, 2, 2)
    plt.imshow(transgenic.Y >= 0.5, cmap='binary')
    plt.title(f'Transgenic - Дифференцировка (покрытие: {transgenic.calculate_coverage():.1f}%, ветвления: {transgenic.count_branches()})')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    compare_models()

